In [17]:
from sys import path
path.append("/home/ec2-user/SageMaker/data-science-development/utils")
path.append("/home/ec2-user/SageMaker/data-science-development/config")

import pandas as pd
import numpy as np

from tqdm import tqdm 
from heapq import nlargest
from collections import Counter

tqdm.pandas()

In [2]:
df_pred = pd.read_csv("../Data/df_pred_ext.csv").drop("Unnamed: 0", axis=1)

In [3]:
df_pred.head()

,candidate_id,job_order,time_between,time_spent,isco_functie_niveau,source,education,company_name,function_id,isco_code4
0,84556,0,37,156,2.0,0,0.0,324258,936,208
1,84556,1,23,116,1.0,0,0.0,324258,809,348
2,84556,2,23,275,2.0,0,0.0,329244,936,208
3,84556,3,1155,4,1.0,0,0.0,368140,1519,344
4,84556,4,203,11,1.0,0,0.0,419826,1519,344


In [4]:
career_paths = df_pred.groupby("candidate_id")["isco_code4"]

In [5]:
# Convert to 2d-arrays, getting rid of candidate_ids as values
career_paths = career_paths.progress_apply(lambda x: x.values)

100%|██████████| 469568/469568 [00:11<00:00, 40221.84it/s]


In [6]:
career_lens = career_paths.apply(len)
career_paths = career_paths.loc[(career_lens > 1)]

In [7]:
career_paths = career_paths.loc[career_paths.apply(lambda x: x[-1] != x[-2])]

In [8]:
career_paths_train = career_paths.iloc[:int(0.8 * 113724)]
career_paths_test = career_paths.iloc[int(0.8 * 113724):]

In [19]:
class_counts = Counter(np.concatenate(career_paths_train.values))

In [66]:
majority_class = set(nlargest(20, class_counts, key=class_counts.get))

In [67]:
majority_class

{117,
 177,
 178,
 185,
 189,
 193,
 195,
 199,
 208,
 222,
 229,
 323,
 326,
 332,
 335,
 343,
 344,
 345,
 348,
 351}

In [68]:
majority_class_performance = career_paths_test.apply(lambda x: x[-1] in majority_class).mean()

In [69]:
majority_class_performance, 1.96 * np.sqrt(((1 - majority_class_performance)* (majority_class_performance)) / (113724*0.2))

(0.6383820619916465, 0.006244250690152155)

In [51]:
print(f"Majority class predictor accuracy: {majority_class_performance * 100:>.2f}%")

Majority class predictor accuracy: 63.84%
